In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense , LSTM, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
import pandas_datareader.data as pdr
import datetime
%load_ext tensorboard
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
import json


In [3]:
class StockDataset():
    ### X_frame, y_frames = 시퀀스 랭스 정하는 변수. 
    def __init__(self, symbol, x_frames, y_frames, start, end):
    
    ### StockDataset('028050.KS', 10, 5, (2001,1,1),(2005,1,1))로 삼성전자 호출. 
        
        self.symbol = symbol
        self.x_frames = x_frames ## sequnce. 
        self.y_frames = y_frames ## 우리는 하루 예측이라서 항상 1 
        
        self.start = datetime.datetime(*start) ##날짜받기.
        self.end = datetime.datetime(*end) ##날짜받기. 

        self.data = pdr.DataReader(self.symbol, 'yahoo', self.start, self.end) ##야후에서 panda로 데이터를 보내줌. 
        print("number of MAN")
        print(self.data.isna().sum())
        
        ###ADj Close Drop out
        self.data = self.data.drop(['Adj Close'], axis=1)
    
    ## sotfmax로 학습하기위해 y값 원핫 인코딩,
    def For_one_hot_train(self,train_rate):
        if 0<train_rate and train_rate<1:
            self.data_train = self.data[ : int(self.data.shape[0]*train_rate)].copy()
            ###스케일 넘파이로 바뀜 !
            Scaler=MinMaxScaler()
            self.data_train = Scaler.fit_transform(self.data_train)
            ###트레인으로 짜름. 

            self.X_data_train = []
            self.y_data_train = []

            #### one_hot 트레이닝 셋으로 만들기. 
            for i in range(self.x_frames, self.data_train.shape[0]):

                self.X_data_train.append(self.data_train[i-self.x_frames : i]) ## 예를들어 x데이터 60일치 짜르기. 


                if self.data_train[i,3] >= self.data_train[i-1,3]: ## 3번쨰 칼럼은 종가를 뜻함. y데이터는 종가보다 올랏으면 1 떨어졌으면 0
                    self.y_data_train.append(1)
                else:
                    self.y_data_train.append(0)

            self.X_data_train= np.array(self.X_data_train)
            self.y_data_train= np.array(self.y_data_train)

            
            ## X_data_train dim = (traing_set, sequence(X_frames) , colummns(x의 요소 ))
            return self.X_data_train, self.y_data_train
        
    def For_one_hot_test(self,test_rate):
        if 0<test_rate and test_rate<1:
            
            
            self.data_test = self.data[int(self.data.shape[0]*(1-test_rate)) : ].copy()
            ###스케일
            Scaler=MinMaxScaler()
            self.data_test = Scaler.fit_transform(self.data_test)
            ###테스트으로 짜름. 

            self.X_data_test = []
            
            self.y_data_test = []

            #### one_hot 테스트 셋으로 만들기. 
            for i in range(self.x_frames, self.data_test.shape[0]):

                self.X_data_test.append(self.data_test[i-self.x_frames : i]) ## 예를들어 x데이터 60일치 짜르기.
                                                                            ## for문에서 마지막 포함 안하고 자르기에서 포함안해서 크기에서 -2 만큼 뒤에. 


                if self.data_test[i,3] >= self.data_test[i-1,3]: ## 3번쨰 칼럼은 종가를 뜻함. y데이터는 종가보다 올랏으면 1 떨어졌으면 0
                    self.y_data_test.append(1)
                else:
                    self.y_data_test.append(0)

            self.X_data_test= np.array(self.X_data_test)
            self.y_data_test= np.array(self.y_data_test)

            return self.X_data_test, self.y_data_test
        
    ###하루예측 dim 맞춰주기. (# of training, # of sequence, # of x의 요소 )
    def For_tommorow(self):
        Scaler=MinMaxScaler()
        self.data_for_tommorow = self.data[self.data.shape[0]-self.x_frames :self.data.shape[0]]
        self.data_for_tommorow = Scaler.fit_transform(self.data_for_tommorow)
        self.data_for_tommorow = np.array(self.data_for_tommorow).reshape(1,self.x_frames,-1)
        return self.data_for_tommorow
        
        
### 로지스틱 정확도 함수 (label 이 두개일 때 . )        
def accuracy(y_pred,y_test):
    y_pred[y_pred<0.5] =0
    y_pred[y_pred>=0.5]=1
    
    result =0
    for i in range(0,y_pred.shape[0]):
        if y_pred[i] == y_test[i]:
            result +=1
    
    accuracy = (result/y_pred.shape[0])*100
    
    return accuracy
    
            

    
        
        
    
        

In [6]:
dataset = StockDataset('005930.KS', 90, 5, (2001,1,1),(2020,11,21))  ##005930.KS ,028050.KS

dataset.data

number of MAN
High         0
Low          0
Open         0
Close        0
Volume       0
Adj Close    0
dtype: int64


,High,Low,Open,Close,Volume
Date,,,,,
2001-01-01,3160.0,3160.0,3160.0,3160.0,0.0
2001-01-02,3380.0,3140.0,3160.0,3350.0,39455000.0
2001-01-03,3420.0,3220.0,3230.0,3370.0,37855000.0
2001-01-04,3870.0,3660.0,3790.0,3780.0,119910000.0
2001-01-05,3920.0,3700.0,3760.0,3900.0,78135000.0
...,...,...,...,...,...
2020-11-16,66700.0,63900.0,64000.0,66300.0,36354334.0
2020-11-17,67000.0,65600.0,67000.0,65700.0,30204089.0
2020-11-18,66200.0,64700.0,65700.0,64800.0,22963790.0


In [4]:
X_train, y_train =dataset.For_one_hot_train(0.8)
X_test, y_test = dataset.For_one_hot_test(0.2)
X_for_tommorow = dataset.For_tommorow()



In [5]:
##모델. 
regression = Sequential()
regression.add(LSTM(units = 60, activation ='relu', return_sequences = True,
                    input_shape=(X_train.shape[1],X_train.shape[2])))
regression.add(Dropout(0.1))


### 마지막 아웃풋 many to one 모델. 
regression.add(LSTM(units = 60, activation ='relu')) ###마지막 아웃풋이라 시퀀스 리턴이 필요가없음. 아웃풋 1나와야됨. 
regression.add(Dropout(0.1))

### 마지막 AL을 다시 fully nn 에 연결. 로지스틱이므로 마지막은 시그모이드로 출력
regression.add(Dense(units = 1))
regression.add(layers.Activation(activation='sigmoid'))

regression.summary()  ### [batche, 인풋, 아웃풋]

## amdam으로 초적화 하고 loss 함수는 label 이 2개임으로 binary_crossentorpy를 이용하였음

 
regression.compile(optimizer='adam', loss='binary_crossentropy')


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 90, 60)            15840     
_________________________________________________________________
dropout (Dropout)            (None, 90, 60)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 60)                29040     
_________________________________________________________________
dropout_1 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 61        
_________________________________________________________________
activation (Activation)      (None, 1)                 0         
Total params: 44,941
Trainable params: 44,941
Non-trainable params: 0
____________________________________________________

In [6]:
### training 시작. 

regression.fit(X_train,y_train,epochs=1, batch_size=32, shuffle=True)

122/122 [==============================] - 8s 69ms/step - loss: 0.6922


In [7]:
##예측값. 
y_pred_train = regression.predict(X_train)
y_pred_test = regression.predict(X_test)
y_pred_tommorow = regression.predict(X_for_tommorow)
up = y_pred_tommorow[0,0]
down = 1- y_pred_tommorow[0,0]
y_pred_tommorow



array([[0.53871346]], dtype=float32)

In [8]:
probability = {"probability" :{"up": str(up),"down": str(down)}}
probability

{'probability': {'up': '0.53871346', 'down': '0.4612865447998047'}}

In [9]:
##정확도 계산과 내일 몇 % 오를지 측정. 
print("train accuracy : %s %%" % accuracy(y_pred_train,y_train))
print("test accuracy : %s %%" % accuracy(y_pred_test,y_test))
print("up : %s %%, down : %s %%" % (up*100, down*100))## 종목 내일의 확률


train accuracy : 53.40005132152938 %
test accuracy : 54.79603087100331 %
up : 53.87134552001953 %, down : 46.12865447998047 %


In [10]:
probability_json = json.dumps(str(probability), indent = 4)
probability_json

'"{\'probability\': {\'up\': \'0.53871346\', \'down\': \'0.4612865447998047\'}}"'

In [11]:
########### 데이터베이스에 올리기. 

cred = credentials.Certificate("./stock-b69a2-firebase-adminsdk-t55a0-66534d7508.json")
##키 파일
firebase_admin.initialize_app(cred, {'databaseURL':'https://fir-76481.firebaseio.com'})
dir = db.reference()
## 데이터베이스 접근. 



In [13]:
dir.update({'소프트웨어 공학':'과제' })

In [12]:
dir.update(probability)
